#### Bring in Required Libraries

In [72]:
import pandas as pd
import pyodbc
import sqlalchemy.types
from icecream import ic
from sqlalchemy import create_engine
from urllib import parse
from timeit import default_timer as timer

In [73]:
# ODBC Connection for FileMaker
start = timer()
cnxn = pyodbc.connect('DSN=FM Clamp ODBC;UID=FMODBC;PWD=FMODBC')
cnxn.timeout = 60
cursor_fm = cnxn.cursor()
print("FileMaker Connect Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Connect Time = 10.222 sec


In [74]:
# SQLAlchemy connection for SQL Server
server = 'tn-sql'
database = 'autodata'
driver = 'ODBC+Driver+17+for+SQL+Server'
user = 'production'
pwd = parse.quote_plus("Auto@matics")
port = '1433'
database_conn = f'mssql+pyodbc://{user}:{pwd}@{server}:{port}/{database}?driver={driver}'
# Make Connection
engine = create_engine(database_conn)
# conn = engine.raw_connection()
conn_sql = engine.connect()

In [75]:
# ODBC Connection for AS400
CONNAS400 = """
Driver={iSeries Access ODBC Driver};
system=10.143.12.10;
Server=AS400;
Database=PROD;
UID=SMY;
PWD=SMY;
"""
dbcnxn = pyodbc.connect(CONNAS400)
cursor_as400 = dbcnxn.cursor()

In [93]:
sql_spares = """
    SELECT PROD.FPSPRMAST1.SPH_PART,
        STRIP(PROD.FPSPRMAST1.SPH_ENGPRT),
        STRIP(PROD.FPSPRMAST1.SPH_DESC1),
        STRIP(PROD.FPSPRMAST1.SPH_DESC2),
        STRIP(PROD.FPSPRMAST1.SPH_MFG),
        STRIP(PROD.FPSPRMAST1.SPH_MFGPRT),
        STRIP(PROD.FPSPRMAST2.SPD_CABINT),
        STRIP(PROD.FPSPRMAST2.SPD_DRAWER),
        PROD.FPSPRMAST2.SPD_QOHCUR,
        PROD.FPSPRMAST1.SPH_CURSTD,
        STRIP(PROD.FPSPRMAST2.SPD_REODTE),
        STRIP(PROD.FPSPRMAST2.SPD_USECC),
        STRIP(PROD.FPSPRMAST2.SPD_PURCC),
        STRIP(PROD.FPSPRMAST2.SPD_QREORD)
    FROM PROD.FPSPRMAST1 INNER JOIN PROD.FPSPRMAST2 ON PROD.FPSPRMAST1.SPH_PART = PROD.FPSPRMAST2.SPD_PART
    WHERE (((PROD.FPSPRMAST2.SPD_FACIL)=9))
"""

In [94]:
sql_parts = """
    SELECT Ourpart,"Band A Part Number", "Housing A Part Number",
        "Screw Part Number" AS Screw, "Band Feed from Band data",
        "Ship Diam Max", "Ship Diam Min", "Hex Size", "Band_Thickness", "Band_Width",
        "CameraInspectionRequired", "ScrDrvChk"
    FROM tbl8Tridon
"""

In [95]:
sql_bands = """
    SELECT "Band Part Number", "Number of Notches",
        "Band Length", "Feed Length", "Description"
    FROM BANDS
"""

### Execute Query on AS400

In [96]:
start = timer()
cursor_as400.execute(sql_spares)
result_spares = cursor_as400.fetchall()
print("AS400 Connect/Query Time = " + str(round((timer() - start), 3)) + " sec")

AS400 Connect/Query Time = 2.117 sec


#### Build Dataframe for Spare Parts

In [100]:
data_type_dict = {'StandardCost' : float, 'OnHand' : int, 'PartNum' : str, 'ReOrderPt' : int}
df_spares = pd.DataFrame.from_records(result_spares)
df_spares.columns = ['PartNum', 'EngPartNum', 'Desc1', 'Desc2', 'Mfg', 'MfgPn', 'Cabinet', 'Drawer', 'OnHand', 'StandardCost','ReOrderDate', 'DeptUse', 'DeptPurch', 'ReOrderPt']
df_spares = df_spares.dropna()
df_spares = df_spares.astype(data_type_dict)
df_spares = df_spares.convert_dtypes()
df_spares.head

<bound method NDFrame.head of        PartNum   EngPartNum                     Desc1  \
0         1056    091001056           PLATE WEAR SIDE   
1         1107    091001107           PLATE SIDE WEAR   
2         1114    091001114     PLATE SIDE WEAR FRONT   
3         1115    091001115           PLATE WEAR SIDE   
4         2001      4-40 H3  SP PT GUN TAP PLUG STYLE   
...        ...          ...                       ...   
22522  1041029   104-10-029              1ST FORM PAD   
22523  3031116  3031-10-016             CUTOFF INSERT   
22524  3031204  3031-20-004            CUT OFF INSERT   
22525  1786196    091786196    SCPW55-14-22  COUPLING   
22526   824091   091824091E  QS186EBQ8  SENSOR EMITER   

                         Desc2 Mfg        MfgPn Cabinet Drawer  OnHand  \
0                                                    C7     D1       4   
1                                                     '      '       0   
2                                                     '      ' 

#### Execute Query on SQL Server

In [97]:
# Clamp Data
start = timer()
cursor_fm.execute(sql_parts)
result_clamp = cursor_fm.fetchall()
print("FileMaker Clamp Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Clamp Query Time = 3.078 sec


In [87]:
# Band Data
start = timer()
cursor_fm.execute(sql_bands)
result_band = cursor_fm.fetchall()
print("FileMaker Band Query Time = " + str(round((timer() - start), 3)) + " sec")

FileMaker Band Query Time = 1.478 sec


#### Build Dataframe for Clamp Data

In [102]:
df_clamps = pd.DataFrame.from_records(result_clamp)
# Set Column Names
df_clamps.columns = ['PartNumber', 'Band', 'Housing', 'Screw', 'Feed', 'DiaMax', 'DiaMin', 'HexSz', 'BandThickness','BandWidth', 'CamInspect', 'ScrDrvChk']
# Set Data Types
data_type_dict = {'PartNumber' : str, 'Band' : str,'Housing' : str,'Screw' : str, 'Feed' : float, 'DiaMax' : float,'DiaMin' : float,'HexSz' : str,'BandThickness' : float, 'BandWidth' : float,'CamInspect' : str,'ScrDrvChk' : str}

# Due Some Filtering and Data Cleansing
df_clamps = df_clamps[df_clamps.Feed != 'N/A']
df_clamps = df_clamps[1:]
df_clamps = df_clamps.astype(data_type_dict)
df_clamps['CamInspect'] = df_clamps['CamInspect'].str.upper()
df_clamps['ScrDrvChk'] = df_clamps['ScrDrvChk'].str.upper()
df_clamps['Feed'] = df_clamps['Feed'].round(3)
df_clamps['BandWidth'] = df_clamps['BandWidth'].round(3)
df_clamps['BandThickness'] = df_clamps['BandThickness'].round(3)
df_clamps.fillna({'DiaMax' : 0.0}, inplace=True)
df_clamps.fillna({'DiaMin' : 0.0}, inplace=True)
df_clamps['DiaMax'] = df_clamps['DiaMax'].round(3)
df_clamps['DiaMin'] = df_clamps['DiaMin'].round(3)
df_clamps = df_clamps.dropna()
df_clamps = df_clamps.convert_dtypes()
df_clamps.head



<bound method NDFrame.head of        PartNumber     Band  Housing      Screw    Feed  DiaMax  DiaMin  HexSz  \
1      M714006102  1042006  2703025    1213002     3.8     0.0    0.83  5/16"   
2      M674029102  1083029  2367012    1278001    8.41     0.0     0.0  5/16"   
3      M674047102  1083047  2367012    1278001  12.165     0.0     0.0  5/16"   
4      M679064102  2019064  2367012  1278001MC    14.5     0.0     0.0  5/16"   
5      M674088102  1083088  2367012    1278001  19.988     0.0     0.0  5/16"   
...           ...      ...      ...        ...     ...     ...     ...    ...   
12151  M665030006  2147030  2157003    2173001    8.41     0.0     2.2  5/16"   
12152   368004175  1081004  4097020  4208010MC   2.625     0.0    0.54   1/4"   
12153   368010175  1081010  4097020  4208010MC   4.083     0.0     1.0   1/4"   
12154   368016175  1081016  4097020  4208010MC   5.424     0.0    1.44   1/4"   
12155  M368024175  1081024  4097020  4208010MC   6.994     0.0    1.44   1/4"  

#### Build Dataframe for Band Data

In [103]:
df_bands = pd.DataFrame.from_records(result_band)
df_bands = df_bands[1:]
df_bands.columns = ['Band', 'Notches', 'BandLength', 'FeedLength', 'Description']
data_type_dict = {'Band' : str, 'Notches' : int,'BandLength' : float,'FeedLength' : float, 'Description' : str}
df_bands = df_bands.dropna()
df_bands = df_bands.astype(data_type_dict)
df_bands = df_bands.convert_dtypes()
df_bands.head(20)

,Band,Notches,BandLength,FeedLength,Description
1,2652012,38,5.043,5.071,"Band 1/2"", Poland"
2,2652013,41,5.244,5.272,"Band 1/2"", Poland"
3,2652014,42,5.433,5.469,"Band 1/2"", Poland"
4,2709012,22,5.688,5.75,"Band Extended Tang 1/2"" Setup"
5,2019065,12,14.438,14.5,Band Slotted
6,2019054,37,13.288,13.35,Band Slotted
7,2009040,34,10.34,10.37,"Band 1/2"" Setup"
8,1229024,22,7.861,7.95,Band Micro Extented Tan
9,1208052,34,12.656,12.718,Band Quick Release
10,1160148,68,31.443,31.505,REG. BAND SETUP


#### Load Spare Parts to SQL Server Table

In [101]:
data_type_dict = {'StandardCost' : sqlalchemy.types.FLOAT, 'OnHand' : sqlalchemy.types.INT,'PartNum' : sqlalchemy.types.VARCHAR(255),'ReOrderPt' : sqlalchemy.types.INT, 'EngPartNum' : sqlalchemy.types.VARCHAR(255), 'Desc1' : sqlalchemy.types.VARCHAR(255),'Desc2' : sqlalchemy.types.VARCHAR(255),'Mfg' : sqlalchemy.types.VARCHAR(255),'MfgPn' : sqlalchemy.types.VARCHAR(255), 'Cabinet' : sqlalchemy.types.VARCHAR(255),'Drawer' : sqlalchemy.types.VARCHAR(255),'ReOrderDate' : sqlalchemy.types.VARCHAR(255),'DeptUse' : sqlalchemy.types.VARCHAR(255),'DeptPurch' : sqlalchemy.types.VARCHAR(255)}

df_spares.to_sql('tblSpares', conn_sql, schema='eng', if_exists='replace', index=False, dtype=data_type_dict)

28

#### Load Clamp to SQL Server Table

In [99]:
data_type_dict = {'PartNumber' : sqlalchemy.types.VARCHAR(255), 'Band' : sqlalchemy.types.VARCHAR(255),'Housing' : sqlalchemy.types.VARCHAR(255),'Screw' : sqlalchemy.types.VARCHAR(255), 'Feed' : sqlalchemy.types.Float, 'DiaMax' : sqlalchemy.types.Float,'DiaMin' : sqlalchemy.types.Float,'HexSz' : sqlalchemy.types.VARCHAR(255),'BandThickness' : sqlalchemy.types.Float, 'BandWidth' : sqlalchemy.types.Float,'CamInspect' : sqlalchemy.types.VARCHAR(255),'ScrDrvChk' : sqlalchemy.types.VARCHAR(255)}

df_clamps.to_sql('parts_clamps', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

32

#### Load Band Data to SQL Server Table

In [92]:
data_type_dict = {'Band' : sqlalchemy.types.VARCHAR(255), 'Notches' : sqlalchemy.types.INT,'BandLength' : sqlalchemy.types.FLOAT,'FeedLength' : sqlalchemy.types.FLOAT, 'Description' : sqlalchemy.types.VARCHAR(255)}

df_bands.to_sql('parts_bands', conn_sql, schema='production', if_exists='replace', index=False, dtype=data_type_dict)

367